In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to your dataset folder
data_dir = "/content/drive/MyDrive/Animal Classification/dataset"


# Check if folders are accessible
import os
print(os.listdir(data_dir))



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
img_size = 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # 80% training, 20% validation
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    "/content/drive/MyDrive/Animal Classification/dataset",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    "/content/drive/MyDrive/Animal Classification/dataset",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

# Load base model (pretrained on ImageNet)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze pretrained layers
for layer in base_model.layers:
    layer.trainable = False

# Build final model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

# Compile
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Train the model

In [ ]:
# 1️⃣ Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,   # you can increase to 15–20 if GPU time allows
    verbose=1
)

# 2️⃣ Batch-wise prediction for faster evaluation
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

y_pred = []
y_true = []

# Ensure your validation generator has shuffle=False
# val_data = ImageDataGenerator(...).flow_from_directory(..., shuffle=False)

for i in range(len(val_data)):
    x_batch, y_batch = val_data[i]            # get batch
    y_pred_batch = model.predict(x_batch)     # predict batch
    y_pred.append(y_pred_batch)
    y_true.append(np.argmax(y_batch, axis=1))

# Stack all batches
y_pred = np.vstack(y_pred)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.hstack(y_true)

# 3️⃣ Get class labels
class_labels = list(val_data.class_indices.keys())

# 4️⃣ Classification report
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# 5️⃣ Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(12,8))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# 6️⃣ Optional: Plot training history
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss")
plt.legend()
plt.show()



In [ ]:
train_dir = "/content/drive/MyDrive/Animal Classification/dataset/train"
val_dir = "/content/drive/MyDrive/Animal Classification/dataset/val"




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = "/content/drive/MyDrive/Animal Classification/dataset"

# Create generators with split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 80% train, 20% val

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)


Build your CNN model

In [ ]:
from tensorflow.keras import layers, models

# Build a simple CNN
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(15, activation='softmax')   # 15 classes
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Train the model

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,   # you can increase to 15–20 if GPU time allows
    verbose=1
)


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Make sure your validation generator has shuffle=False
# val_data = ImageDataGenerator(...).flow_from_directory(..., shuffle=False)

# 1️⃣ Batch-wise predictions
y_pred = []
y_true = []

for i in range(len(val_data)):
    x_batch, y_batch = val_data[i]
    y_pred_batch = model.predict(x_batch)
    y_pred.append(y_pred_batch)
    y_true.append(np.argmax(y_batch, axis=1))

y_pred = np.vstack(y_pred)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.hstack(y_true)

# 2️⃣ Class labels
class_labels = list(val_data.class_indices.keys())

# 3️⃣ Classification report
print("Classification Report:\n")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# 4️⃣ Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(12,8))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# 5️⃣ Plot training history
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss")
plt.legend()
plt.show()


Fix the overfitting

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


Data Augmentation

In [ ]:

data_dir = "/content/drive/MyDrive/Animal Classification/dataset"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)



Build Model using MobileNetV2

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze base layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),          # prevent overfitting
    layers.Dense(15, activation='softmax')  # 15 classes
])
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,   # can increase to 15–20
    verbose=1
)

Evaluate and Plot

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Batch-wise predictions
y_pred = []
y_true = []

for i in range(len(val_data)):
    x_batch, y_batch = val_data[i]
    y_pred_batch = model.predict(x_batch)
    y_pred.append(y_pred_batch)
    y_true.append(np.argmax(y_batch, axis=1))

y_pred = np.vstack(y_pred)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.hstack(y_true)

class_labels = list(val_data.class_indices.keys())

# Classification report
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(12,8))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# Plot training history
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss")
plt.legend()
plt.show()


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-50]:  # freeze first layers, fine-tune last 50
    layer.trainable = False

model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,  # fine-tuning usually takes fewer epochs
    verbose=1
)


In [ ]:
# Batch-wise predictions
y_pred = []
y_true = []

for i in range(len(val_data)):
    x_batch, y_batch = val_data[i]
    y_pred_batch = model.predict(x_batch)
    y_pred.append(y_pred_batch)
    y_true.append(np.argmax(y_batch, axis=1))

y_pred = np.vstack(y_pred)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.hstack(y_true)

class_labels = list(val_data.class_indices.keys())

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print(classification_report(y_true, y_pred_classes, target_names=class_labels))

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(12,8))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


Save your fine-tuned model for inference:

In [ ]:
model.save("/content/drive/MyDrive/Animal Classification/fine_tuned_mobilenetv2.h5")


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Direct path from your Google Drive
img_path = "/content/drive/MyDrive/Animal Classification/dataset/Bear/Bear_10.jpg"

# Load and preprocess the image
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)/255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)
class_index = np.argmax(pred)
class_labels = list(val_data.class_indices.keys())
print("Predicted Class:", class_labels[class_index])


In [ ]:
# Evaluate on the validation set
val_loss, val_acc = model.evaluate(val_data)
print("Validation Accuracy:", val_acc)
print("Validation Loss:", val_loss)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
root_folder = "/content/drive/MyDrive/Animal Classification/TestImages/"


In [ ]:
import os
print(os.path.exists(root_folder))  # Should print True


In [ ]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Root folder containing subfolders (Bear, Cat, etc.)
root_folder = "/content/drive/MyDrive/Animal Classification/dataset/TestImages/"

class_labels = list(val_data.class_indices.keys())

# Collect all image paths from subfolders
img_files = []
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)
    if os.path.isdir(subfolder_path):
        for f in os.listdir(subfolder_path):
            if f.endswith(('.jpg', '.png')):
                img_files.append(os.path.join(subfolder_path, f))

# Predict and visualize
plt.figure(figsize=(20, 10))
for i, img_path in enumerate(img_files):
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    class_index = np.argmax(pred)

    plt.subplot(4, int(np.ceil(len(img_files)/4)), i+1)  # 4 rows, adjust columns automatically
    plt.imshow(img)
    plt.title(class_labels[class_index])
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import os

root_folder = "/content/drive/MyDrive/Animal Classification/TestImages/"  # Adjust if needed
print(os.path.exists(root_folder))  # Should print True


In [ ]:
print(os.listdir(root_folder))  # Should list subfolders like Bear, Cat, etc.


In [ ]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Root folder containing subfolders (Bear, Cat, etc.)
root_folder = "/content/drive/MyDrive/Animal Classification/TestImages/"

class_labels = list(val_data.class_indices.keys())

# Collect all image paths from subfolders along with true labels
img_files = []
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)
    if os.path.isdir(subfolder_path):
        for f in os.listdir(subfolder_path):
            if f.endswith(('.jpg', '.png')):
                img_files.append((os.path.join(subfolder_path, f), subfolder))  # Save true label

# Initialize counters
correct = 0
total = len(img_files)

batch_size = 20  # Number of images per figure

# Predict, visualize, and compute accuracy in batches
for start in range(0, total, batch_size):
    end = start + batch_size
    plt.figure(figsize=(20, 10))

    for i, (img_path, true_label) in enumerate(img_files[start:end]):
        img = image.load_img(img_path, target_size=(224,224))
        img_array = image.img_to_array(img)/255.0
        img_array = np.expand_dims(img_array, axis=0)

        pred = model.predict(img_array)
        class_index = np.argmax(pred)
        predicted_label = class_labels[class_index]

        if predicted_label == true_label:
            correct += 1

        plt.subplot(4, int(np.ceil(batch_size/4)), i+1)
        plt.imshow(img)
        plt.title(f"P: {predicted_label}\nT: {true_label}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Compute overall accuracy
accuracy = correct / total
print(f"Overall TestImages Accuracy: {accuracy*100:.2f}%")

